In [1]:
import json
import pandas as pd
import re
import sys
from pathlib import Path
sys.path.append(str(Path("../src/").resolve()))
from desenhar_automatos import desenhar_afn, desenhar_afd


In [2]:
afe = {
  "estados": ["q0", "q1", "q2", "q3", "q4", "q5", "q6", "qf"],
  "alfabeto": ["a", "b", "c"],
  "estado_inicial": "q0",
  "estados_finais": ["qf"],
  "transicoes": {
    "q0": {
      "a": ["q0"],
      "b": ["q0"],
      "c": ["q0"],
      "ε": ["q1", "q2", "q4"]
    },
    "q1": {
      "a": ["qf"],
      "b": [],
      "c": [],
      "ε": []
    },
    "q2": {
      "a": [],
      "b": ["q3"],
      "c": [],
      "ε": []
    },
    "q3": {
      "a": [],
      "b": ["qf"],
      "c": [],
      "ε": []
    },
    "q4": {
      "a": [],
      "b": [],
      "c": ["q5"],
      "ε": []
    },
    "q5": {
      "a": [],
      "b": [],
      "c": ["q6"],
      "ε": []
    },
    "q6": {
      "a": [],
      "b": [],
      "c": ["qf"],
      "ε": []
    },
    "qf": {
      "a": [],
      "b": [],
      "c": [],
      "ε": []
    }
  }
}


# AFe - AFN

In [3]:
df_transicoes = pd.DataFrame(afe["transicoes"]).T
df_transicoes.columns = ["a","b","c","ε"]
df_transicoes

def fechoEpsilon(estado, fechamento=None):
    
    if fechamento is None:
        fechamento = set()
    
    fechamento.add(estado)

    proximos_estados = df_transicoes.T[estado]["ε"]
    
    for s in proximos_estados:
        if s not in fechamento:
            fechoEpsilon(s, fechamento)
    
    return fechamento

def novoDelta(qi, x):
    deltaEpsilonQi = fechoEpsilon(qi)
    conjunto_x_mv = [set(df_transicoes[x][s]) for s in deltaEpsilonQi]
    conjunto_transicoes_x = set().union(*conjunto_x_mv)
    conjunto_x_epsilon = [fechoEpsilon(r) for r in conjunto_transicoes_x]
    novas_transicoes_x = set().union(*conjunto_x_epsilon)
    return list(novas_transicoes_x)

# Conjunto de estados finais
estados_finais = set(afe["estados_finais"])

# Novas transições
novas_transicoes_AFN = {}
novos_estados_finais = set()

for t in df_transicoes.index.to_list():
    novas_transicoes_AFN[t] = {}
    for a in afe["alfabeto"]:
        novas_transicoes_AFN[t][a] = novoDelta(t, a)

    if fechoEpsilon(t) & estados_finais:
        novos_estados_finais.add(t)

# eAFN, modelo antigo
afn = afe.copy()

# Atualização do modelo
afn["transicoes"] = novas_transicoes_AFN
afn["estados_finais"] = list(novos_estados_finais)


In [4]:
afn

{'estados': ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'qf'],
 'alfabeto': ['a', 'b', 'c'],
 'estado_inicial': 'q0',
 'estados_finais': ['qf'],
 'transicoes': {'q0': {'a': ['q4', 'q0', 'qf', 'q1', 'q2'],
   'b': ['q4', 'q0', 'q1', 'q2', 'q3'],
   'c': ['q4', 'q0', 'q5', 'q1', 'q2']},
  'q1': {'a': ['qf'], 'b': [], 'c': []},
  'q2': {'a': [], 'b': ['q3'], 'c': []},
  'q3': {'a': [], 'b': ['qf'], 'c': []},
  'q4': {'a': [], 'b': [], 'c': ['q5']},
  'q5': {'a': [], 'b': [], 'c': ['q6']},
  'q6': {'a': [], 'b': [], 'c': ['qf']},
  'qf': {'a': [], 'b': [], 'c': []}}}

# AFN - AFD

In [6]:
def gera_pi(pi, transicao): # Ex.: ('q0',)

    novo_pi = set()
    
    for estado in pi:
        qiqu = set(df_transicoes.T[estado][transicao])
        novo_pi = novo_pi.union(qiqu)

    novo_pi_l = list(novo_pi)
    novo_pi_l.sort()

    novo_pi = tuple(novo_pi_l)

    return novo_pi

def gera_transicoes(pi, pilha=None, novas_transicoes=None):
   
    if pilha is None:
        pilha = set()

    if novas_transicoes is None:
        novas_transicoes = {}
        
    if pi in pilha:
        return novas_transicoes
    else:
        pilha.add(pi)
        
    novas_transicoes[pi] = {}
    for transicao in df_transicoes.T.index.to_list():
        novo_pi = gera_pi(pi, transicao)
        novas_transicoes[pi][transicao] = novo_pi
        gera_transicoes(novo_pi, pilha, novas_transicoes)

    return novas_transicoes    

# Para q0 => q0q1 de uma transição a
p1 = tuple(df_transicoes.T["q0"]["a"])

# Criação do conjunto de estados 'pi'
_pi_ = set()
_pi_.add(p1)
_pi_

estado_inicial = ('q0',)
nt = gera_transicoes(estado_inicial)
nt

# Função que faz a redução de '('q0', 'q1')' => 'q0q1'
ct = lambda s: re.sub(r"[()',\s]", "", str(s))

# DataFrame criando a partir de um novo objeto reduzido e válido
df_nova_transicoes = pd.DataFrame.from_dict({
    
    ct(index): {
        transicao: ct(estado)            
        
        for transicao, estado in value.items()
    }
    
    for index, value in nt.items()
    
}, orient="index")


# Dicionário para tradução dos novos estados
dtraduz = {
    estado: f"p{i}*" if any([qf in estado for qf in afn["estados_finais"]]) else f"p{i}"
    for i, estado in enumerate(df_nova_transicoes.index.to_list())
}

AFD_transicoes = df_nova_transicoes.rename(index=dtraduz).replace(dtraduz)

# Recuperando estados finais
AFD_estados_finais = [estado for _, estado in dtraduz.items() if '*' in estado]

# Montando modelo por partes
afd = afn.copy()
afd["estados"] = AFD_transicoes.index.to_list()
afd["estado_inicial"] = AFD_transicoes.index[0]
afd["estados_finais"] = AFD_estados_finais
afd["transicoes"] = AFD_transicoes.T.to_dict()

In [7]:
afd

{'estados': ['p0', 'p1', 'p2*', 'p3', 'p4', 'p5', 'p6'],
 'alfabeto': ['a', 'b', 'c'],
 'estado_inicial': 'p0',
 'estados_finais': ['p2*'],
 'transicoes': {'p0': {'a': 'p0', 'b': 'p0', 'c': 'p0', 'ε': 'p1'},
  'p1': {'a': 'p2*', 'b': 'p4', 'c': 'p5', 'ε': 'p3'},
  'p2*': {'a': 'p3', 'b': 'p3', 'c': 'p3', 'ε': 'p3'},
  'p3': {'a': 'p3', 'b': 'p3', 'c': 'p3', 'ε': 'p3'},
  'p4': {'a': 'p3', 'b': 'p2*', 'c': 'p3', 'ε': 'p3'},
  'p5': {'a': 'p3', 'b': 'p3', 'c': 'p6', 'ε': 'p3'},
  'p6': {'a': 'p3', 'b': 'p3', 'c': 'p2*', 'ε': 'p3'}}}

# Minimização

In [8]:
def minimizar_via_tabela(afd):
    estados = sorted(afd['estados'])
    finais = set(afd['estados_finais'])
    transicoes = afd['transicoes']
    alfabeto = afd['alfabeto']

    # PASSO A - Construir Tabela
    tabela = {}
    pares = []
    for i in range(len(estados)):
        for j in range(i + 1, len(estados)):
            u, v = estados[i], estados[j]
            tabela[(u, v)] = False
            pares.append((u, v))


    # PASSO B - Estado final ou não final
    for (u, v) in pares:
        u_final = u in finais
        v_final = v in finais
        if u_final != v_final:
            tabela[(u, v)] = True

    # PASSO C - Criação/ Comparação de pares
    mudou = True
    while mudou:
        mudou = False
        for (u, v) in pares:
            if not tabela[(u, v)]:
                for simbolo in alfabeto:
                    dest_u = transicoes.get(u, {}).get(simbolo)
                    dest_v = transicoes.get(v, {}).get(simbolo)

                    if dest_u and dest_v and dest_u != dest_v:
                        p_dest = tuple(sorted((dest_u, dest_v)))
                          
                        if tabela.get(p_dest, False):
                            tabela[(u, v)] = True
                            mudou = True
                            break 

    # PASSO D - Unificar estados equivalentes
    parent = {e: e for e in estados}
    def find(n):
        if parent[n] != n: parent[n] = find(parent[n])
        return parent[n]
    def union(n1, n2):
        root1, root2 = find(n1), find(n2)
        if root1 != root2: parent[root2] = root1

    for (u, v), marcado in tabela.items():
        if not marcado:
            union(u, v)

    grupos = {}
    for e in estados:
        root = find(e)
        grupos.setdefault(root, []).append(e)
    
    mapa_nomes = {e: "_".join(sorted(lista)) for root, lista in grupos.items() for e in lista}

    novos_estados = sorted(list(set(mapa_nomes.values())))
    novo_inicial = mapa_nomes[afd['estado_inicial']]
    novos_finais = sorted(list(set(mapa_nomes[e] for e in finais)))
    
    novas_transicoes = {}
    for root, lista in grupos.items():
        rep = lista[0]
        nome_grupo = mapa_nomes[rep]
        novas_transicoes[nome_grupo] = {}
        for s in alfabeto:
            destino_original = transicoes.get(rep, {}).get(s)
            if destino_original:
                novas_transicoes[nome_grupo][s] = mapa_nomes[destino_original]

    afd_unificado = {
        "estados": novos_estados,
        "alfabeto": alfabeto,
        "estado_inicial": novo_inicial,
        "estados_finais": novos_finais,
        "transicoes": novas_transicoes
    }

    # PASSO E - Remover estados inuteis
    alcancaveis = set()
    fila = [afd_unificado['estado_inicial']]
    while fila:
        curr = fila.pop(0)
        if curr not in alcancaveis:
            alcancaveis.add(curr)
            for s in alfabeto:
                prox = afd_unificado['transicoes'].get(curr, {}).get(s)
                if prox: fila.append(prox)

    reverso = {e: [] for e in afd_unificado['estados']}
    for orig, trans in afd_unificado['transicoes'].items():
        for s, dest in trans.items():
            if dest in reverso: reverso[dest].append(orig)
    
    vivos = set()
    fila_vivos = [e for e in afd_unificado['estados_finais']] 
    visited_vivos = set(fila_vivos)
    
    while fila_vivos:
        curr = fila_vivos.pop(0)
        vivos.add(curr)
        for pai in reverso.get(curr, []):
            if pai not in visited_vivos:
                visited_vivos.add(pai)
                fila_vivos.append(pai)
    
    estados_uteis = sorted([e for e in alcancaveis if e in vivos])
    
    trans_final = {
        e: {s: d for s, d in afd_unificado['transicoes'][e].items() if d in estados_uteis}
        for e in estados_uteis
    }

    return {
        "estados": estados_uteis,
        "alfabeto": alfabeto,
        "estado_inicial": novo_inicial if novo_inicial in estados_uteis else None,
        "estados_finais": [e for e in novos_finais if e in estados_uteis],
        "transicoes": trans_final
    }


In [9]:
mini = minimizar_via_tabela(afd)
mini

{'estados': [],
 'alfabeto': ['a', 'b', 'c'],
 'estado_inicial': None,
 'estados_finais': [],
 'transicoes': {}}

# Resultados

In [ ]:
desenhar_afn(afe, "AFNε")

In [ ]:
desenhar_afn(afn, "AFN")

# ---

In [ ]:
# --- EXECUÇÃO ---

automato_input = {
  "estados": ["q0", "q1", "q2", "q3", "q4", "q5"],
  "alfabeto": ["a", "b"],
  "estado_inicial": "q0",
  "estados_finais": ["q3"],
  "transicoes": {
    "q0": {"a": "q1", "b": "q2"},
    "q1": {"a": "q3", "b": "q4"},
    "q2": {"a": "q3", "b": "q4"},
    "q3": {"a": "q3", "b": "q3"},
    "q4": {"a": "q4", "b": "q4"},
    "q5": {"a": "q0", "b": "q1"}
  }
}

afd_minimizado = minimizar_via_tabela(automato_input)

In [ ]:
desenhar_afd(automato_input, "Autômato Original (Antes)")

In [ ]:
desenhar_afd(afd_minimizado, "Autômato Minimizado (Passos A-E)")